In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [98]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras.utils import plot_model

from tensorflow import set_random_seed
set_random_seed(20)

In [99]:
# making real data
#data come from : http://idojarasbudapest.hu/archivalt-idojaras
# by hand collected data
#path2  = "https://drive.google.com/drive/my-drive/ido1_2019.txt"
path = "U:\\deeep\\ido1_2019.xlsx"
df = pd.read_excel(path,  sheet_name="2019")
df.head()

,Nap,Tmax[°C],Tmin[°C],Szél[m/s],Csapadék[mm]
0,2019-01-01 00:00:00,NaN,NaN,NaN,NaN
1,kedd,2.7,-4.6,6.1,0.9
2,2019-01-02 00:00:00,NaN,NaN,NaN,NaN
3,szerda,3.1,-0.7,9.5,0.9
4,2019-01-03 00:00:00,NaN,NaN,NaN,NaN


In [100]:
# backfilling the data to the date rows
df.fillna(method='bfill', inplace=True)
df.head()

,Nap,Tmax[°C],Tmin[°C],Szél[m/s],Csapadék[mm]
0,2019-01-01 00:00:00,2.7,-4.6,6.1,0.9
1,kedd,2.7,-4.6,6.1,0.9
2,2019-01-02 00:00:00,3.1,-0.7,9.5,0.9
3,szerda,3.1,-0.7,9.5,0.9
4,2019-01-03 00:00:00,0.3,-3.3,7.9,0.1


In [101]:
# clear the rows of the days
df = df.loc[df.index %2 == 0]
df.head()

,Nap,Tmax[°C],Tmin[°C],Szél[m/s],Csapadék[mm]
0,2019-01-01 00:00:00,2.7,-4.6,6.1,0.9
2,2019-01-02 00:00:00,3.1,-0.7,9.5,0.9
4,2019-01-03 00:00:00,0.3,-3.3,7.9,0.1
6,2019-01-04 00:00:00,-1.0,-6.3,5.4,0.2
8,2019-01-05 00:00:00,2.2,-3.2,5.8,4.0


In [102]:
df.set_index('Nap', inplace=True) 
df.tail()

,Tmax[°C],Tmin[°C],Szél[m/s],Csapadék[mm]
Nap,,,,
2019-10-24,22.1,14.8,2.8,0.0
2019-10-25,21.9,14.7,2.3,0.0
2019-10-26,19.7,13.7,1.7,0.0
2019-10-27,20.4,13.6,1.2,0.0
2019-10-28,17.2,12.3,4.0,0.0


In [103]:
#Making a treain-test datasets
X = df.copy()
# the target is the average of the maximal and minimal tempratures
y = df['Tmax[°C]'] + df['Tmin[°C]'] / 2


splitfactor = 1/3
split = int(len(X)*splitfactor)
# splitting the data by hand at 666
X_tr, y_tr = X[:-split], y[:-split]
X_val, y_val = X[split:], y[split:]


in_shape = X_tr.shape # store the number of features
X_tr.shape, y_tr.shape

((201, 4), (201,))

In [109]:
# building the model
mod = Sequential()
#layer1- input
out1 = 12
mod.add(Dense(out1, input_shape=(X_tr.shape[1],)))  #  the output 12 neuron
mod.add(Activation('relu')) # with relu 

#layer2
out2 = 12
mod.add(Dense(out2)) # output has 12 neuron
mod.add(Activation('relu'))

#layer3
out3 = 1
mod.add(Dense(out3)) 
mod.add(Activation('relu'))

# compile , but not learn
mod.compile(loss='mse', optimizer=SGD(lr=0.1, momentum=0.001), metrics=['mse'])
mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 12)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 12)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 12)                156       
_________________________________________________________________
activation_17 (Activation)   (None, 12)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 13        
_________________________________________________________________
activation_18 (Activation)   (None, 1)                 0         
Total params: 229
Trainable params: 229
Non-trainable params: 0
_________________________________________________________________


In [110]:
from keras.callbacks import ReduceLROnPlateau
reduceLR = ReduceLROnPlateau(monitor='val_score', patient=12, factor=0.75, min_lr=0.001)

In [111]:
batchs = int(np.sqrt(in_shape[0]))
mod.fit(X, y, batch_size=batchs, epochs=50, callbacks=[reduceLR], verbose=1, shuffle=True)


Epoch 1/50
301/301 [==============================] - 2s 6ms/step - loss: 781.7523 - mean_squared_error: 781.7523
Epoch 2/50
301/301 [==============================] - 0s 361us/step - loss: 781.4033 - mean_squared_error: 781.4033
Epoch 3/50
112/301 [==========>...................] - ETA: 0s - loss: 812.7636 - mean_squared_error: 812.7636

U:\anaconda3\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_score` which is not available. Available metrics are: loss,mean_squared_error,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


301/301 [==============================] - 0s 633us/step - loss: 781.4033 - mean_squared_error: 781.4033
Epoch 4/50
301/301 [==============================] - 0s 444us/step - loss: 781.4033 - mean_squared_error: 781.4033
Epoch 5/50
301/301 [==============================] - 0s 331us/step - loss: 781.4033 - mean_squared_error: 781.4033
Epoch 6/50
301/301 [==============================] - 0s 302us/step - loss: 781.4033 - mean_squared_error: 781.4033
Epoch 7/50
301/301 [==============================] - 0s 302us/step - loss: 781.4033 - mean_squared_error: 781.4033
Epoch 8/50
301/301 [==============================] - 0s 302us/step - loss: 781.4033 - mean_squared_error: 781.4033
Epoch 9/50
301/301 [==============================] - 0s 311us/step - loss: 781.4033 - mean_squared_error: 781.4033
Epoch 10/50
301/301 [==============================] - 0s 351us/step - loss: 781.4033 - mean_squared_error: 781.4033
Epoch 11/50
301/301 [==============================] - 0s 364us/step - loss: 781.4

In [112]:
mod.evaluate(X_val, y_val, batch_size=batchs, verbose=1)

201/201 [==============================] - 0s 1ms/step


[1105.7350227014342, 1105.7350227014342]